In [15]:
prob.variables()

[MOT_MOTeight,
 MOT_MOTeleven,
 MOT_MOTfive,
 MOT_MOTfour,
 MOT_MOTfourteen,
 MOT_MOTnine,
 MOT_MOTseven,
 MOT_MOTsix,
 MOT_MOTten,
 MOT_MOTthirteen,
 MOT_MOTthree,
 MOT_MOTtwelve,
 MOT_MOTtwo,
 VTO_VTOeight,
 VTO_VTOeleven,
 VTO_VTOfive,
 VTO_VTOfour,
 VTO_VTOfourteen,
 VTO_VTOnine,
 VTO_VTOseven,
 VTO_VTOsix,
 VTO_VTOten,
 VTO_VTOthirteen,
 VTO_VTOthree,
 VTO_VTOtwelve,
 VTO_VTOtwo,
 backlog_backlogWeek10,
 backlog_backlogWeek11,
 backlog_backlogWeek12,
 backlog_backlogWeek13,
 backlog_backlogWeek14,
 backlog_backlogWeek2,
 backlog_backlogWeek3,
 backlog_backlogWeek4,
 backlog_backlogWeek5,
 backlog_backlogWeek6,
 backlog_backlogWeek7,
 backlog_backlogWeek8,
 backlog_backlogWeek9,
 dummy1_dummyWeek10,
 dummy1_dummyWeek11,
 dummy1_dummyWeek12,
 dummy1_dummyWeek13,
 dummy1_dummyWeek14,
 dummy1_dummyWeek2,
 dummy1_dummyWeek3,
 dummy1_dummyWeek4,
 dummy1_dummyWeek5,
 dummy1_dummyWeek6,
 dummy1_dummyWeek7,
 dummy1_dummyWeek8,
 dummy1_dummyWeek9,
 dummy2_dummyWeek10,
 dummy2_dummyWeek11,
 

In [14]:
dummyVariableNamesList5

['dummyWeek2',
 'dummyWeek3',
 'dummyWeek4',
 'dummyWeek5',
 'dummyWeek6',
 'dummyWeek7',
 'dummyWeek8',
 'dummyWeek9',
 'dummyWeek10',
 'dummyWeek11',
 'dummyWeek12',
 'dummyWeek13',
 'dummyWeek14']

In [6]:
#####################################################################
########      Creating initial parameters         ###################
#####################################################################
import matplotlib.pyplot as plt
%matplotlib inline 
from __future__ import division 
from pandasql import sqldf
import pandas as pd
import copy
import numpy as np
import inflect
from pulp import *  

# extract from a dataframe using a filter on week the desirable field 
def extract(data,week,field):
    return float(data[data.week == week][field])    
# a more general function to extract field2 data based on filtering on field1
def extract2(data,field1,condition,field2):
    return float(data[data[field1] == condition][field2])  

def plotter(objectList):
    fig, ax = plt.subplots(1, sharex=False, sharey=False)
    data = [(x, y.varValue) for x in objectList for y in prob.variables() if y.name.split('_')[1] == x ]
    fig.set_size_inches(30, 5)
    my_xticks = [i[0] for i in data]
    plt.xticks(range(len(data)), my_xticks)
    labels = ax.get_xticklabels()
    ax.set_xticklabels(labels, rotation=-30)
    plt.plot([i[1] for i in data])
    plt.show()
    
    
# run this at the end of the first optimizaiton when still current week is 1 
def headcountDfUpdater(currentWeek,headcountDf):
    # solving in the current week determines the hires 4 weeks from now
    # hiring decision for the first 3 weeks is frozen 
    week = currentWeek+3
    # converts numerical week to string week, for example 1 becomes 'one'
    weekWord = str(inflect.engine().number_to_words(week).replace(" ", ""))
    hireType = ['AMZ','TEMP']
    hireDate = []
    headcountValue = []
    laborType = []
    for hireTypeIntsnace in hireType:
        employeeTarget = 'employee' + hireTypeIntsnace + weekWord
        for v in prob.variables():
            # if the decision variable name is 'employeeAMZfour' then add the value to the table 
            emplpyeeDecisionVariableName = str(v.name.split('_')[1])
            if emplpyeeDecisionVariableName == employeeTarget:
                hireDate.append(week)
                laborType.append(hireTypeIntsnace)
                headcountValue.append(float(v.varValue))
    newHeadcountDf = pd.DataFrame(np.column_stack((hireDate,headcountValue,laborType)),
                                    columns=['hireDate', 'headcountValue','laborType' ])
    frames = [headcountDf, newHeadcountDf]
    headcountDf = pd.concat(frames)
    hireDate2Integer = lambda x: int(x['hireDate'])
    headcountDf['hireDate'] = headcountDf.apply(hireDate2Integer,axis = 1)
    return headcountDf

# run this at the end of the first optimizaiton when still current week is 1 
def BacklogUpdater(currentWeek,backlogDf):
    for v in prob.variables():
        DecisionVariableNames = str(v.name).split('_')
        if (DecisionVariableNames[0] == 'backlog') & (DecisionVariableNames[1] == 'backlogWeek' + str(currentWeek)):
            week = [currentWeek]
            backlogValue = [v.varValue]
    newBacklogDf = pd.DataFrame(np.column_stack((week,backlogValue)),
                                columns=['week', 'backlogValue' ])
    frames = [backlogDf, newBacklogDf]
    BacklogDf = pd.concat(frames)
    backlogDf['week'] = backlogDf.apply(week2Integer,axis = 1)
    return BacklogDf

def labor_cost(currentWeek):
    weekString = str(inflect.engine().number_to_words(currentWeek).replace(" ", "")) 
    laborCost = 0
    for v in prob.variables():
        DecisionVariableNames = str(v.name).split('_')
        if (DecisionVariableNames[0] == 'MOT') & (DecisionVariableNames[1] == 'MOT' + weekString):
            laborCost += (+1)*v.varValue*MOTClassNamesDict['MOT' + weekString].cost
        if (DecisionVariableNames[0] == 'VTO') & (DecisionVariableNames[1] == 'VTO' + weekString):
            laborCost += (-1)*v.varValue*VTOClassNamesDict['VTO' + weekString].cost
    exec("laborCost += currentEmployeeCostWeek" + str(currentWeek))
    return laborCost

simulationHorizon = 26 # weeks to generate values 
simulations = ['attrition']
runs = 1
counter = 0 
costs = {}
costs2 = list()
backlogGraph = list()
hiringGraph = list()
solution = list()
for simulation in simulations:
#     costs = {}
    percentAttendance = np.linspace(0,0,runs)
    percentAttrition = np.linspace(0,0,runs)
    percentDemand = np.linspace(1,1,runs) 
    percentRate = np.linspace(1,1,runs) 
    if simulation == 'attendance':
        percentAttendance = np.linspace(0.02,-0.02,runs)
    if simulation == 'attrition':
        percentAttrition = np.linspace(-0.02,0.02,runs)
    if simulation == 'demand':
        percentDemand = np.linspace(.98,1.02,runs)
    if simulation == 'rate':
        percentRate = np.linspace(1.02,0.98,runs)
    for run in range(runs):
        print('atten',percentAttendance[run],'attr',percentAttrition[run],'dem',percentDemand[run],'rat',percentRate[run])
        counter += 1
        prob = ''
        print('run ',run,' for ', simulation)
        
        week =  list(range(-4,simulationHorizon)) + list(range(-4,simulationHorizon))
        headcount = np.linspace(100,100,len(week))
#         attritionCount = [8]*int(len(week)/2) + [1]*int(len(week)/2)
        attritionCount = [10]*int(len(week)/2) + [3]*int(len(week)/2)
        laborType = ['TEMP']*int(len(week)/2) + ['AMZ']*int(len(week)/2)
        attritionActuals = pd.DataFrame(np.column_stack((week,headcount,attritionCount,laborType)),
                                        columns=['week','headcount', 'attritionCount',  'laborType'])
        attritionRateCalculator = lambda x: float(x['attritionCount'])/float(x['headcount'])
        attritionActuals['actualRate'] = attritionActuals.apply(attritionRateCalculator,axis = 1)
        week2Integer = lambda x: int(x['week'])
        attritionActuals['week'] = attritionActuals.apply(week2Integer,axis = 1)
        pysqldf = lambda q: sqldf(q, globals())
        # attrition inputs 
        attritionForecastsDf =  pysqldf("""
        SELECT 
         a.week
        ,a.laborType
        ,a.attritionCount
        ,a.headcount
        ,a.actualRate
        ,SUM(b.attritionCount)/SUM(b.headcount) AS predictedRate
        FROM attritionActuals a 
        LEFT JOIN attritionActuals b
        ON (a.laborType = b.laborType)
        AND b.week between (a.week - 4) and (a.week - 1) 
        GROUP BY a.week,a.laborType
        order by a.week ; """)
    #     print(attritionForecastsDf)
    # made sure the rates do not go below 0. 
        attritionForecastsDf.predictedRate = attritionForecastsDf.predictedRate + percentAttrition[run]

        hoursScheduled = [100]*int(len(week)/2) + [100]*int(len(week)/2)
        hoursWorked =  [70.0]*int(len(week)/2) + [80.0]*int(len(week)/2)
        laborType = ['TEMP']*int(len(week)/2) + ['AMZ']*int(len(week)/2)
        attendanceActuals = pd.DataFrame(np.column_stack((week,hoursWorked,hoursScheduled,laborType)),
                                        columns=['week', 'hoursWorked', 'hoursScheduled','laborType' ])
        attendanceRateCalculator = lambda x: float(x['hoursWorked'])/float(x['hoursScheduled'])
        attendanceActuals['actualRate'] = attendanceActuals.apply(attendanceRateCalculator,axis = 1)
        attendanceActuals['week'] = attendanceActuals.apply(week2Integer,axis = 1)
        pysqldf = lambda q: sqldf(q, globals())
        # attendance inputs 
        attendanceForecastsDf =  pysqldf("""
        SELECT 
         a.week
        ,a.laborType
        ,a.hoursWorked
        ,a.hoursScheduled
        ,a.actualRate
        ,SUM(b.hoursWorked)/SUM(b.hoursScheduled) AS predictedRate
        ,a.actualRate -  SUM(b.hoursWorked)/SUM(b.hoursScheduled) as error
        FROM attendanceActuals a 
        LEFT JOIN attendanceActuals b
        ON (a.laborType = b.laborType)
        AND b.week between (a.week - 4) and (a.week - 1) 
        GROUP BY a.week,a.laborType
        order by a.week ; """)
        attendanceForecastsDf.predictedRate = attendanceForecastsDf.predictedRate + percentAttendance[run]
    #     print(attendanceForecastsDf)
        # learning curve rates 
        learningCurveValues = list(np.linspace(0.65,1.00,int(len(week)/2))) + list(np.linspace(0.65,1.00,int(len(week)/2)))
        laborType = ['TEMP']*int(len(week)/2) + ['AMZ']*int(len(week)/2)
        learningCurveDf = pd.DataFrame(np.column_stack((week,learningCurveValues,laborType)),
                                        columns=['week', 'learningCurveValue','laborType' ])
        learningCurveDf['week'] = learningCurveDf.apply(week2Integer,axis = 1)
        # veteran rates and labor cost 
        laborCost = [15,20]
        laborType = ['TEMP','AMZ']
        veteranRateValues = [131*percentRate[run] ,131*percentRate[run]]
        laborTypeDf = pd.DataFrame(np.column_stack((laborType,veteranRateValues,laborCost)),
                                        columns=['laborType' ,'veteranRate','laborCost'])
        # current employee headcount and their hiring dates 
        hireDate =  list(range(-10,4)) + list(range(-10,4))
        headcountValue = np.linspace(15,15,len(hireDate))
        headcountValue[0] = 10  # amazonian veterans 
        laborType =  ['AMZ']*int(len(hireDate)/2) + ['TEMP']*int(len(hireDate)/2) 
        headcountDf = pd.DataFrame(np.column_stack((hireDate,headcountValue,laborType)),
                                        columns=['hireDate', 'headcountValue','laborType' ])
        hireDate2Integer = lambda x: int(x['hireDate'])
        headcountDf['hireDate'] = headcountDf.apply(hireDate2Integer,axis = 1)
#         print(headcountDf)
        # demand 
        week =  range(-4,simulationHorizon)
    #     demandActual = np.linspace(2000000,2000000,len(week))
        np.random.seed(1234)
        demandActual = np.linspace(1000000*percentDemand[run],2500000*percentDemand[run],len(week))
        demandForecast = demandActual + np.random.normal(0,demandActual[0]*0.1,len(week))*0 
        
        demandDf = pd.DataFrame(np.column_stack((week,demandActual,demandForecast)),
                                        columns=['week', 'demandActual','demandForecast' ])
        demandDf['week'] = demandDf.apply(week2Integer,axis = 1)
        # initial backlog  
        backlogValue = [0]
        week = [0]
        backlogDf = pd.DataFrame(np.column_stack((week,backlogValue)),
                                        columns=['week', 'backlogValue' ])
        backlogDf['week'] = backlogDf.apply(week2Integer,axis = 1)



    #####################################################################
    ###############  Creating Decision Variable Calsses #################
    #####################################################################
    # hireDate = the hiring date (week number) of the employee group of interest 
    # targetWeek = the date at which the survival value is requested
    # currentWeek = the week in which the calculation is made (this determines what prediction to use)
        def survivalDataFrame(currentWeek,attritionForecastsDf):
            attritionUpdatedDf =  pysqldf("""
                SELECT 
                 a.week
                ,a.laborType
                ,a.attritionCount
                ,a.headcount
                ,a.actualRate
                ,CASE 
                    WHEN a.week <= """ + str(currentWeek) + """ THEN a.actualRate
                    ELSE a.predictedRate """ + str(improvement) + """*a.error  as predictedRate
                FROM attritionForecastsDf
                ON (a.laborType = b.laborType)
                AND b.week between (a.week - 4) and (a.week - 1) 
                GROUP BY a.week,a.laborType
                order by a.week ; """) 
            return attritionUpdatedDf
    
    # laborType = temp or amz 
        def survivalFunc(hireDate,targetWeek,currentWeek,laborType): # prediction depends on currentWeek
            data = attritionForecastsDf[attritionForecastsDf.laborType == laborType]
            if hireDate <= 0:  # we do not apply attrition to weeks prior to week 1
                if targetWeek <= 0:
                    return 1
                if targetWeek > 0:
                    return (1- data[data.week == currentWeek].predictedRate)**(targetWeek)
            if hireDate <= targetWeek :
                # prediction value depend on currentWeek , compounding depends on targetWeek  
                return (1- data[data.week == currentWeek].predictedRate)**(targetWeek - hireDate + 1)
            if hireDate > targetWeek :
                return 0

        def attendanceFunc(hireDate,targetWeek,currentWeek,laborType): # prediction depends on currentWeek 
            data = attendanceForecastsDf[attendanceForecastsDf.laborType == laborType]
            if hireDate <= 0:  # attendance prior to week 1 is not considered - do not call for values prior to week 1
                if targetWeek <= 0:
                    return 1
                if targetWeek > 0:
                    return data[data.week == currentWeek].predictedRate
            if hireDate <= targetWeek :
                return data[data.week == currentWeek].predictedRate
            if hireDate > targetWeek :
                return 0


        def learningCurveFunc(hireDate,targetWeek,laborType):  # rate does not depend on current week 
                data = learningCurveDf[learningCurveDf.laborType == laborType]
                if targetWeek - hireDate > 10:
                    return 1
                if targetWeek < hireDate: # if you have not been hired yet, no learning curve 
                    return 0
                else:
                    return float(data[data.week == int(targetWeek - hireDate)].learningCurveValue)

        def veteranRateFunc(laborType):  # rate does not depend on current week 
                return float(laborTypeDf[laborTypeDf.laborType == laborType].veteranRate)    

        # class employee keeps track of a cohort information through time using methods 
        class employee():
            Schedulehours = 40
            def __init__(self,laborType = '',hireDate = 0,regularCost = 0,initialHeadcount = 0,mainProcess = [],currentWeek = 1, name = 'empty'):
                self.laborType = laborType
                self.hireDate = hireDate
                self.regularCost = regularCost
                self.initialHeadcount = initialHeadcount
                self.mainProcess = mainProcess
                self.currentWeek = currentWeek
                self.name = name
            def effectiveSurvival(self,targetWeek):
                return float(survivalFunc(self.hireDate,targetWeek,self.currentWeek,self.laborType))

            def effectiveAttendance(self,targetWeek):
                return float(attendanceFunc(self.hireDate,targetWeek,self.currentWeek,self.laborType))

            def effectiveHeadcountCoef(self,targetWeek):
                survived = self.effectiveSurvival(targetWeek)
                attended = self.effectiveAttendance(targetWeek)
                return float(survived*attended)        

            def effectiveHeadcount(self,targetWeek):
                E = self.effectiveHeadcountCoef(targetWeek)
                return float(self.initialHeadcount*E)

            def showHoursCoef(self,targetWeek):
                return float(self.effectiveHeadcountCoef(targetWeek)*self.Schedulehours)

            def showHours(self,targetWeek):
                return float(self.showHoursCoef(targetWeek)*self.initialHeadcount)

            def learningCurve(self,targetWeek):
                return learningCurveFunc(self.hireDate,targetWeek,self.laborType)

            def veteranRate(self):
                return float(veteranRateFunc(self.laborType))
            
            def rate(self,targetWeek):
                return float(self.learningCurve(targetWeek)*self.veteranRate())

            def capacityCoef(self,targetWeek):
                showHourCoefValue = self.showHoursCoef(targetWeek)
                learningCurveValue = self.learningCurve(targetWeek)
                veteranRateValue = self.veteranRate()
                return float(showHourCoefValue*learningCurveValue*veteranRateValue)

            def capacity(self,targetWeek):
                return float(self.capacityCoef(targetWeek)*self.initialHeadcount)

            def costCoef(self,targetWeek):
                if self.hireDate <= 0: 
                    if targetWeek <= 0:
                        return 0
                    if targetWeek > 0:
                        return float(self.regularCost*self.showHoursCoef(targetWeek))
                if self.hireDate > targetWeek:
                    return 0
                if self.hireDate <= targetWeek:
                    return float(self.regularCost*self.showHoursCoef(targetWeek))

            def cost(self,targetWeek):
                return float(self.costCoef(targetWeek)*self.initialHeadcount)


            def vetCoef(self,targetWeek):
                if targetWeek - 10 >= self.hireDate :
                    return 1
                else:
                    return 0

            def vetHoursCoef(self,targetWeek):
                return self.vetCoef(targetWeek)*self.showHoursCoef(targetWeek)

            def vetHours(self,targetWeek):
                return self.vetCoef(targetWeek)*self.showHours(targetWeek)

            def vetCapacity(self,targetWeek):
                return self.vetHours(targetWeek)*self.veteranRate()
    # embassadors spent 1 full day with new hires, then follow up with 5 of them for up to 2 days 
            def trainingCapacityCoef(self,targetWeek):
                if self.hireDate == targetWeek and self.hireDate > 0:
                    vetTraining = 3*self.veteranRate()  # we lose 3 vet hour of vets capacity per new hire  
                    newHireTraining = self.learningCurve(targetWeek)*self.veteranRate()*2 #2 hours of new hire went on safety training 
                    return float(vetTraining+newHireTraining)
                else:
                    return 0  #current head-count do not need training , training only occures in week 1
 #connect this to the prevoious method 
            def trainingVetHourCoef(self,targetWeek):
                if self.hireDate == targetWeek and self.hireDate > 0:
                    vetTraining = 3 # we lose 1 hour of vets per new hire 
                    return float(vetTraining)
                else:
                    return 0  #current head-count do not need training , training only occures in week 1

        class MOT():
            cost = (extract2(laborTypeDf,'laborType','AMZ','laborCost')+
                    extract2(laborTypeDf,'laborType','TEMP','laborCost'))*(1.5/2)  # average times 1.5 
            rate = extract2(laborTypeDf,'laborType','AMZ','veteranRate')
            def __init__(self,name ='', targetWeek = 0):
                self.name = name 
                self.targetWeek= targetWeek  # the week you call MOT
            def hourIndicator(self,week):
                if self.targetWeek == week:
                    return 1
                else: 
                    return 0 

        class VTO():
            cost = extract2(laborTypeDf,'laborType','TEMP','laborCost')
            rate = extract2(laborTypeDf,'laborType','AMZ','veteranRate')    
            def __init__(self,name ='', targetWeek = 0):
                self.name = name 
                self.targetWeek = targetWeek   # the week you call VTO
            def hourIndicator(self,week):
                if self.targetWeek == week:
                    return 1
                else: 
                    return 0 


        class laborShare(): 
            def __init__(self,name ='', targetWeek = 0):
                self.name = name 
                self.targetWeek = targetWeek 
            def hourIndicator(self,week):
                if self.targetWeek == week:
                    return 1
                else: 
                    return 0 



# # #####################################################################
# # ###############  ROUND 2 OPTIMIZATION WEEK 2 #################
# # #####################################################################
# # ### ********** to do ******* #######
# totalCost = 0
# # #####################################################################
# # ###############        creating class instances      ################
# # #####################################################################
# for simulationWeeks in range(1,3):
#     print('running optimization for week {} .... '.format(simulationWeeks))
#     prob = ''
#     # the current week or the week when we are solving for labor
#     currentWeek = simulationWeeks

#     # for current employees we have their headcount
#     employeeStatus = ['current','future']
#     for employeeStatusItem in employeeStatus:
#         code = compile( employeeStatusItem + 'Employee' + 'ClassList = list()',  '<string>', 'exec')  
#         exec(code)

#     # iterate throught the table of current headcount and generate classes 
#     # stores all the employee objects into a list 
#     for index,row in headcountDf[(headcountDf.hireDate <= currentWeek + 2)].iterrows(): 
#         name = """employee""" + str(row.laborType.upper()) + str(inflect.engine().number_to_words(row.hireDate).replace(" ", "")) 
#         code = compile( name  + """ = employee(
#                                             laborType = """ + """str(row.laborType)""" + """,
#                                             hireDate = """ + str(row.hireDate) + """,
#                                             regularCost = """ + str(extract2(laborTypeDf,'laborType',str(row.laborType),'laborCost'))  + """,  
#                                             initialHeadcount = """ + str(row.headcountValue) + """,
#                                             mainProcess = [],
#                                             currentWeek = """ + str(currentWeek) + """,
#                                             name = '""" + name  + """')""",
#                                             '<string>', 'exec')
#         exec(code)
#         code = compile( 'currentEmployeeClassList.append' + '(' +name+ ')',
#                                             '<string>', 'exec')   
#         exec(code)
#     # generat hiring variables for new hires - hiring is freezed in the first 3 weeks  
#     for hireDate in range(currentWeek + 3,currentWeek + 13):
#         for laborType in ['AMZ','TEMP']:
#                 name = """employee""" + str(laborType.upper()) + str(inflect.engine().number_to_words(hireDate).replace(" ", "")) 
#                 code = compile( name  + """ = employee(
#                                                     laborType = laborType,
#                                                     hireDate = """ + str(hireDate) + """,
#                                                     regularCost = """ + str(extract2(laborTypeDf,'laborType',laborType,'laborCost'))  + """,    
#                                                     initialHeadcount = 0,
#                                                     mainProcess = [],
#                                                     currentWeek = """ + str(currentWeek) + """,
#                                                     name = '""" + name  + """')""",
#                                                     '<string>', 'exec')
#                 exec(code)
#                 code = compile( 'futureEmployeeClassList.append' + '(' +name+ ')',
#                                                     '<string>', 'exec')   
#                 exec(code)

#     # generat VTO classes for each week 
#     VTOClassList = list()  # list of objects 
#     for week in range(currentWeek ,currentWeek + 13):
#         for timeLever in ['VTO']:
#             name = timeLever + str(inflect.engine().number_to_words(week).replace(" ", "")) 
#             code = compile( (name  + """ = """ + timeLever +
#                                                 """(name = '""" + name + 
#                                                 """', targetWeek = week)"""),
#                                                 '<string>', 'exec')
#             exec(code)
#             code = compile( 'VTOClassList.append' + '(' +name+ ')',
#                                                 '<string>', 'exec')   
#             exec(code)


#     # generat MOT classes for each week 
#     MOTClassList = list()  # list of objects 
#     for week in range(currentWeek ,currentWeek + 13):
#         for timeLever in ['MOT']:
#             name = timeLever + str(inflect.engine().number_to_words(week).replace(" ", "")) 
#             code = compile( (name  + """ = """ + timeLever +
#                                                 """(name = '""" + name + 
#                                                 """', targetWeek = week)"""),
#                                                 '<string>', 'exec')
#             exec(code)
#             code = compile( 'MOTClassList.append' + '(' +name+ ')',
#                                                 '<string>', 'exec')   
#             exec(code)

#     # generat laborShare classes for each week 
#     laborShareClassList = list()  # list of objects 
#     for week in range(currentWeek ,currentWeek + 13):
#         name = 'laborShare' + str(inflect.engine().number_to_words(week).replace(" ", "")) 
#         code = compile( (name  + """ = laborShare(
#                                             name = '""" + name + """' 
#                                            ,targetWeek = week)"""),
#                                             '<string>', 'exec')
#         exec(code)
#         code = compile( 'laborShareClassList.append' + '(' +name+ ')',
#                                             '<string>', 'exec')
#         exec(code)

#     #####################################################################
#     ##     creating lists of names of the objects created earlier   #####
#     #####################################################################

#     # create a list of names of the instances of classes created earlier 
#     # create a dictionary in which the key is the name of the class instance and the value is the class instance 
#     futureEmployeeClassNamesList = list()
#     futureEmployeeClassNamesDict = {}
#     for variables in futureEmployeeClassList:
#         futureEmployeeClassNamesList.append(variables.name)
#         futureEmployeeClassNamesDict[variables.name] = variables 
#     # create a list of names of the instances of classes created earlier 
#     # create a dictionary in which the key is the name of the class instance and the value is the class instance 
#     currentEmployeeClassNamesList = list()
#     currentEmployeeClassNamesDict = {}
#     for variables in currentEmployeeClassList:
#         currentEmployeeClassNamesList.append(variables.name)
#         currentEmployeeClassNamesDict[variables.name] = variables 
#     # create a list of names of the instances of classes created earlier 
#     # create a dictionary in which the key is the name of the class instance and the value is the class instance 
#     MOTClassNamesList = list()
#     MOTClassNamesDict = {}
#     for variables in MOTClassList:
#         MOTClassNamesList.append(variables.name) 
#         MOTClassNamesDict[variables.name] = variables 
#     # create a list of names of the instances of classes created earlier 
#     # create a dictionary in which the key is the name of the class instance and the value is the class instance         
#     VTOClassNamesList = list()
#     VTOClassNamesDict = {}
#     for variables in VTOClassList:
#         VTOClassNamesList.append(variables.name) 
#         VTOClassNamesDict[variables.name] = variables 
#     # create a list of names of the instances of classes created earlier 
#     # create a dictionary in which the key is the name of the class instance and the value is the class instance         
#     laborShareClassNamesList = list()
#     laborShareClassNamesDict = {}
#     for variables in laborShareClassList:
#         laborShareClassNamesList.append(variables.name)    
#         laborShareClassNamesDict[variables.name] = variables 

#     ################################################################################
#     ##     creating dictionaries capacity, cost for current and futrue employees ###
#     ################################################################################

#     optimizationHorizon = range(currentWeek,currentWeek+13)

#     # creating dictionaries with 
#     # keys = futureEmployeeCapacityWeek + week + decision variable name  
#     # values = coeficient of capacity (h * coeficient = capacity)
#     for week in optimizationHorizon:
#         exec(compile( 'futureEmployeeCapacityWeek' + str(week) + '={}','<string>', 'exec'))
#         for variables in futureEmployeeClassList:
#             exec(compile( 'futureEmployeeCapacityWeek' + str(week) + '[variables.name] = variables.capacityCoef(' + str(week) + ')'                     
#                          ,'<string>', 'exec'))

#     # weekly capacity of current emplpoyees     
#     # keys = currentEmployeeCapacityWeek  +  week
#     # value = sum of all processing capacity of current hires (plus the first three weeks hires)
#     for week in optimizationHorizon:
#         exec(compile( 'currentEmployeeCapacityWeek' + str(week) + '={}','<string>', 'exec'))
#         s = sum([variables.capacity(week) for variables in currentEmployeeClassList])
#         exec(compile( 'currentEmployeeCapacityWeek' + str(week) + ' = s '                   
#                          ,'<string>', 'exec'))


#     # weekly cost of future emplpoyees  
#     # name = per week for example for week 1 a dictionary is generated with name futureEmployeeCostWeek1
#     # keys = employee name  for example employeeAMZfour
#     # value = sum of all processing capacity of current hires (plus the first three weeks hires) on that week 
#     for week in optimizationHorizon:
#         exec(compile( 'futureEmployeeCostWeek' + str(week) + '={}','<string>', 'exec'))
#         for variables in futureEmployeeClassList:
#             exec(compile( 'futureEmployeeCostWeek' + str(week) + '[variables.name] = variables.costCoef(' + str(week) + ')'                     
#                          ,'<string>', 'exec'))

#     # weekly cost of current emplpoyees  
#     # name = per week for example for week 1 a variable is generated with name currentEmployeeCostWeek1
#     # value = sum of all processing capacity of current hires (plus the first three weeks hires) on that week 
#     for week in optimizationHorizon:
#         s = sum([variables.cost(week) for variables in currentEmployeeClassList])
#         exec(compile( 'currentEmployeeCostWeek' + str(week) + ' = s '                   
#                          ,'<string>', 'exec'))

#     # weekly total veteran hour coef of future employees 
#     # there is one dictionary per week with name :  futureEmployeeVeteranHourWeek + week
#     # each dictionary has :
#     # key : employee class name
#     # value : cost of that employee class in the specified week 
#     for week in optimizationHorizon:
#         exec(compile( 'futureEmployeeVeteranHourWeek' + str(week) + '={}','<string>', 'exec'))
#         for variables in futureEmployeeClassList:
#             exec(compile( 'futureEmployeeVeteranHourWeek' + str(week) + '[variables.name] = variables.vetHoursCoef(' + str(week) + ')'                     
#                          ,'<string>', 'exec'))

#     # weekly total veteran hour from current employees 
#     # name = per week for example for week 1 a variable is generated with name urrentEmployeeVeteranHourWeek1
#     # value = sum of all veteran hours from the current employees 
#     for week in optimizationHorizon:
#         s = sum([variables.vetHours(week) for variables in currentEmployeeClassList])
#         exec(compile( 'currentEmployeeVeteranHourWeek' + str(week) + ' = s '                   
#                          ,'<string>', 'exec'))

#     for week in optimizationHorizon:
#         s = sum([variables.effectiveHeadcount(week)*variables.rate(week) for variables in currentEmployeeClassList])
#         s2 = sum([variables.effectiveHeadcount(week) for variables in currentEmployeeClassList])
#         exec(compile( 'currentEmployeeRateWeek' + str(week) + ' = s/s2 '                   
#                          ,'<string>', 'exec'))
#     ###########################################################
#     ##     creating decision variables for optimization    ####
#     ###########################################################

#     futureEmployeeDecisionVariables = LpVariable.dicts("futureEmployee",futureEmployeeClassNamesList,lowBound=0,upBound=100)
#     MOTDecisionVariables = LpVariable.dicts("MOT",MOTClassNamesList,lowBound=0,upBound=10000000)
#     VTODecisionVariables = LpVariable.dicts("VTO",VTOClassNamesList,lowBound=0,upBound=10000000)
#     laborShareDecisionVariables = LpVariable.dicts("laborShare",laborShareClassNamesList,lowBound=0,upBound=10000000)
#     endWeekBacklogVariableNamesList = ['backlogWeek' + str(i) for i in optimizationHorizon]
#     endWeekBacklogDecisionVariables = LpVariable.dicts("backlog",endWeekBacklogVariableNamesList,lowBound=0,upBound=None)    
#     dummyVariableNamesList1 = ['dummy1Week' + str(i) for i in optimizationHorizon]
#     dummyDecisionVariables1 = LpVariable.dicts("dummy1",dummyVariableNamesList1,lowBound=0,upBound=None)    
#     dummyVariableNamesList2 = ['dummy2Week' + str(i) for i in optimizationHorizon]
#     dummyDecisionVariables2 = LpVariable.dicts("dummy2",dummyVariableNamesList2,lowBound=0,upBound=None)    
#     dummyVariableNamesList3 = ['dummy3Week' + str(i) for i in optimizationHorizon]
#     dummyDecisionVariables3 = LpVariable.dicts("dummy3",dummyVariableNamesList3,lowBound=0,upBound=None)    
#     dummyVariableNamesList4 = ['dummy4Week' + str(i) for i in optimizationHorizon]
#     dummyDecisionVariables4 = LpVariable.dicts("dummy4",dummyVariableNamesList4,lowBound=0,upBound=None)  
#     dummyVariableNamesList5 = ['dummy5Week' + str(i) for i in optimizationHorizon]
#     dummyDecisionVariables5 = LpVariable.dicts("dummy5",dummyVariableNamesList5,lowBound=0,upBound=None)  
#     dummyVariableNamesList6 = ['dummy6Week' + str(i) for i in optimizationHorizon]
#     dummyDecisionVariables6 = LpVariable.dicts("dummy6",dummyVariableNamesList6,lowBound=0,upBound=None)  
#     ###################################################
#     ##     Defining the optimization function      ####
#     ###################################################
#     prob = LpProblem("Labor Planning Basic",LpMinimize)
#     ###################################################
#     ##     Defining the objective                  ####
#     ###################################################
# #     print(headcountDf)
#     print('generating objective function ....')
#     exec("""prob += lpSum([futureEmployeeCostWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+1) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+2) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+3) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+4) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+5) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+6) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+7) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+8) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+9) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+10) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+11) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [futureEmployeeCostWeek"""+ str(currentWeek+12) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList ] +
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+1) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+2) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+3) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+4) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+5) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+6) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+7) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+8) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+9) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+10) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+11) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
#                   [globals()[i].hourIndicator(week = """+ str(currentWeek+12) +""")*globals()[i].cost*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +  
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+1) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] + 
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+2) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +    
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+3) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+4) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] + 
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+5) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+6) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] + 
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+7) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+8) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] + 
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+9) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+10) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] + 
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+11) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#                   [globals()[i].hourIndicator(week =  """+ str(currentWeek+12) +""")*globals()[i].cost*MOTDecisionVariables[i] for i in MOTClassNamesList] +  
#                   [0.00001*endWeekBacklogDecisionVariables[i] for i in endWeekBacklogVariableNamesList ] + 
#                   [0.00001*laborShareDecisionVariables[i] for i in laborShareClassNamesList ] + 
#                   [100000000*dummyDecisionVariables1[i] for i in dummyVariableNamesList1] + 
#                   [100000000*dummyDecisionVariables2[i] for i in dummyVariableNamesList2] + 
#                   [100000000*dummyDecisionVariables3[i] for i in dummyVariableNamesList3] + 
#                   [100000000*dummyDecisionVariables4[i] for i in dummyVariableNamesList4] + 
#                   [100000000*dummyDecisionVariables5[i] for i in dummyVariableNamesList5] + 
#                   [100000000*dummyDecisionVariables6[i] for i in dummyVariableNamesList6] +                   
#                   [currentEmployeeCostWeek"""+ str(currentWeek) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+1) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+2) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+3) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+4) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+5) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+6) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+7) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+8) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+9) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+10) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+11) +""" +
#                    currentEmployeeCostWeek"""+ str(currentWeek+12) +""" ]) """)
#     ###################################################
#     ##     Defining the constraints                 ###
#     ###################################################
#     # week 1  
#     # units capacity 
#     print('generating current week constraints ....')
# #     print('capacity week {}'.format(currentWeek))
#     exec("""prob += lpSum(([futureEmployeeCapacityWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].trainingCapacityCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
#                    ))*(1 + 1*0.3/7)  + currentEmployeeCapacityWeek"""+ str(currentWeek) +"""*(1 + 1*0.3/7) +lpSum([dummyDecisionVariables1[v] for v in dummyDecisionVariables1 if int(v.split('Week')[-1])==currentWeek]) >= (
#     extract(demandDf,week="""+ str(currentWeek) +""",field ='demandActual') + extract(backlogDf,week="""+ str(currentWeek-1) +""",field ='backlogValue')), "Capacity Requirement week"""+ str(currentWeek) +""""

#     # units capacity 
# # print('process week {}'.format(currentWeek))
# prob += lpSum(([futureEmployeeCapacityWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].trainingCapacityCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +                   
#                    [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*(-1)*VTODecisionVariables[i] for i in VTOClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
#                    )) + currentEmployeeCapacityWeek"""+ str(currentWeek) +""" <= (
#     extract(demandDf,week="""+ str(currentWeek) +""",field ='demandActual') + extract(backlogDf,week="""+ str(currentWeek-1) +""",field ='backlogValue'))+lpSum([dummyDecisionVariables2[v] for v in dummyDecisionVariables2 if int(v.split('Week')[-1])==currentWeek]) ,  "process less than demand week"""+ str(currentWeek) +""""
#     # units capacity 
# # print('backlog week {}'.format(currentWeek))
# prob += endWeekBacklogDecisionVariables['backlogWeek"""+ str(currentWeek) +"""'] == (
#             extract(demandDf,week="""+ str(currentWeek) +""",field ='demandActual') + extract(backlogDf,week="""+ str(currentWeek-1) +""",field ='backlogValue') + 
#     -1*(lpSum(([futureEmployeeCapacityWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#            [globals()[i].trainingCapacityCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +              
#            [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
#            [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#            [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
#            ))+ currentEmployeeCapacityWeek"""+ str(currentWeek) +""")),  "Backlog flow balance week"""+ str(currentWeek) +""""
#     # units hour
# # print('MOT week {}'.format(currentWeek))
# prob += [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*MOTDecisionVariables[i] for i in MOTClassNamesList] <= (
#             lpSum(([futureEmployeeVeteranHourWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].trainingVetHourCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] 
#                    )) + currentEmployeeVeteranHourWeek"""+ str(currentWeek) +""")*0.1 +lpSum([dummyDecisionVariables3[v] for v in dummyDecisionVariables3 if int(v.split('Week')[-1])==currentWeek]), "week """+ str(currentWeek) +""" MOT constraint  """+ str(currentWeek) +""" "
#     # units hour 
# # print('MOT week {}'.format(currentWeek))
# prob += [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*VTODecisionVariables[i] for i in VTOClassNamesList] <= ( 
#             lpSum(([futureEmployeeVeteranHourWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].trainingVetHourCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList]         
#                    )) + currentEmployeeVeteranHourWeek"""+ str(currentWeek) +""" )*0.1 + lpSum([dummyDecisionVariables4[v] for v in dummyDecisionVariables4 if int(v.split('Week')[-1])==currentWeek]), "week """+ str(currentWeek) +""" VTO constraint week """+ str(currentWeek) +""" "
# # print('laborshare week {}'.format(currentWeek))
# prob += lpSum([globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*laborShareDecisionVariables[i] for i in laborShareClassNamesList]) <= ( 
#             lpSum(([futureEmployeeCapacityWeek"""+ str(currentWeek) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].trainingCapacityCoef(targetWeek = """+ str(currentWeek) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(currentWeek) +""")*currentEmployeeRateWeek"""+ str(currentWeek) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] 
#                    ))*(1) + currentEmployeeCapacityWeek"""+ str(currentWeek) +""" ) +lpSum([dummyDecisionVariables5[v] for v in dummyDecisionVariables5 if int(v.split('Week')[-1])==currentWeek]) , "week """+ str(currentWeek) +""" laborShare constraint """+ str(currentWeek) +""""  """)


#     # week 2 and + constraints 
#     A = ["""
# # print('capacity week {}'.format("""+ str(week) +"""))
# prob += lpSum(([futureEmployeeCapacityWeek"""+ str(week) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].trainingCapacityCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(week) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
#                    ))*(1+0.5/7) + currentEmployeeCapacityWeek"""+ str(week) +"""*(1+0.5/7) +lpSum([dummyDecisionVariables1[v] for v in dummyDecisionVariables1 if int(v.split('Week')[-1])==week]) >= (
#     extract(demandDf,week="""+ str(week) +""",field ='demandForecast') +  endWeekBacklogDecisionVariables['backlogWeek"""+ str(week-1) +"""'] ), "Capacity Requirement week """+ str(week) +""" "

# # print('process week {}'.format("""+ str(week) +"""))
# prob += lpSum(([futureEmployeeCapacityWeek""" +  str(week)  +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].trainingCapacityCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(week) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
#                    )) + currentEmployeeCapacityWeek"""+ str(week) +""" <= (
#     extract(demandDf,week="""+ str(week) +""",field ='demandForecast') +  endWeekBacklogDecisionVariables['backlogWeek"""+ str(week-1) +"""'] ) +lpSum(lpSum([dummyDecisionVariables2[v] for v in dummyDecisionVariables2 if int(v.split('Week')[-1])==week])) ,  "process less than demand week"""+ str(week) +""" "

# # print('backlog week {}'.format("""+ str(week) +"""))
# prob += endWeekBacklogDecisionVariables['backlogWeek"""+ str(week) +"""'] == (
#             extract(demandDf,week="""+ str(week) +""",field ='demandForecast') +  endWeekBacklogDecisionVariables['backlogWeek"""+ str(week-1) +"""'] + 
#     -1*(lpSum(([futureEmployeeCapacityWeek"""+ str(week) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#            [globals()[i].trainingCapacityCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +    
#            [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
#            [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] +
#            [globals()[i].hourIndicator(week = """+ str(week) +""")*(-1)*laborShareDecisionVariables[i] for i in laborShareClassNamesList] 
#            ))+ currentEmployeeCapacityWeek"""+ str(week) +""")),  "Backlog flow balance week"""+ str(week) +""" "

# # print('MOT week {}'.format("""+ str(week) +"""))
# prob += [globals()[i].hourIndicator(week = """+ str(week) +""")*MOTDecisionVariables[i] for i in MOTClassNamesList] <= (
#             lpSum(([futureEmployeeVeteranHourWeek"""+ str(week) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].trainingVetHourCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] 
#                    )) +currentEmployeeVeteranHourWeek"""+ str(week) +""" )*0.1 +lpSum([dummyDecisionVariables3[v] for v in dummyDecisionVariables3 if int(v.split('Week')[-1])==week]), "week """+ str(week) +""" MOT constraint"

# # print('VTO week {}'.format("""+ str(week) +"""))
# prob += [globals()[i].hourIndicator(week = """+ str(week) +""")*VTODecisionVariables[i] for i in VTOClassNamesList] <= ( 
#             lpSum(([futureEmployeeVeteranHourWeek"""+ str(week) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].trainingVetHourCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] 
#                   )) +  currentEmployeeVeteranHourWeek"""+ str(week) +""" )*0.1 +lpSum([dummyDecisionVariables4[v] for v in dummyDecisionVariables4 if int(v.split('Week')[-1])==week]), "week """+ str(week) +""" VTO constraint"

# # print('laborshare week {}'.format("""+ str(week) +"""))
# prob += lpSum([globals()[i].hourIndicator(week = """+ str(week) +""")*laborShareDecisionVariables[i] for i in laborShareClassNamesList]) <= ( 
#             lpSum(([futureEmployeeCapacityWeek"""+ str(week) +"""[i]*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].trainingCapacityCoef(targetWeek = """+ str(week) +""")*(-1)*futureEmployeeDecisionVariables[i] for i in futureEmployeeClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*(-1)*VTODecisionVariables[i] for i in  VTOClassNamesList] +
#                    [globals()[i].hourIndicator(week = """+ str(week) +""")*currentEmployeeRateWeek"""+ str(week) +"""*MOTDecisionVariables[i] for i in MOTClassNamesList] 
#                    ))*(1) + currentEmployeeCapacityWeek"""+ str(week) +""" ) + lpSum([dummyDecisionVariables5[v] for v in dummyDecisionVariables5 if int(v.split('Week')[-1])==week]), "week """+ str(week) +""" laborShare constraint" """ for week in range(currentWeek+1,currentWeek+13)]

#     for i in A:
#         exec(i)

#     prob.solve()
#     costs[run] = value(prob.objective)
#     costs[str(simulation)+ 'run '+str(run)] = value(prob.objective)
#     print("Status:", LpStatus[prob.status])
#     costs2.append((str(counter)+ ''+ str(simulation)+ 'run '+str(run), value(prob.objective)))

#     print("Total Cost  = ", value(prob.objective))
# #     print(str(simulation)+' run '+str(run), "current employee capacity =", int(currentEmployeeCapacityWeek1))
#     #         if simulation == 'rate':


#     plotter(futureEmployeeClassNamesList)
#     plotter(MOTClassNamesList)
#     plotter(endWeekBacklogVariableNamesList)
#     plotter(VTOClassNamesList)
#     plotter(laborShareClassNamesList)
#     plotter(dummyVariableNamesList1)
#     plotter(dummyVariableNamesList2)
#     plotter(dummyVariableNamesList3)
#     plotter(dummyVariableNamesList4)
#     plotter(dummyVariableNamesList5)
#     plotter(backlogDf)
#     print([(i,i.varValue) for i in prob.variables()])
#     print('current week cost {}'.format(labor_cost(currentWeek)))
#     totalCost= totalCost+labor_cost(currentWeek)
#     print('total cost {}'.format(totalCost))
#     headcountDf = headcountDfUpdater(currentWeek,headcountDf)
#     backlogDf = BacklogUpdater(currentWeek,backlogDf)
# #     print('backlog :',backlogDf)

#     solution.append([(i,i.varValue) for i in prob.variables()])

atten 0.0 attr -0.02 dem 1.0 rat 1.0
run  0  for  attrition


In [10]:
attritionForecastsDf

,week,laborType,attritionCount,headcount,actualRate,predictedRate
0,-4,AMZ,3,100.0,0.03,NaN
1,-4,TEMP,10,100.0,0.10,NaN
2,-3,AMZ,3,100.0,0.03,0.01
3,-3,TEMP,10,100.0,0.10,0.08
4,-2,AMZ,3,100.0,0.03,0.01
5,-2,TEMP,10,100.0,0.10,0.08
6,-1,AMZ,3,100.0,0.03,0.01
7,-1,TEMP,10,100.0,0.10,0.08
8,0,AMZ,3,100.0,0.03,0.01
9,0,TEMP,10,100.0,0.10,0.08


In [2]:
!pip install pandasql


You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [60]:
prob.writeLP

<bound method LpProblem.writeLP of Labor Planning Basic:
MINIMIZE
26.25*MOT_MOTeight + 26.25*MOT_MOTeleven + 26.25*MOT_MOTfive + 26.25*MOT_MOTfour + 26.25*MOT_MOTfourteen + 26.25*MOT_MOTnine + 26.25*MOT_MOTseven + 26.25*MOT_MOTsix + 26.25*MOT_MOTten + 26.25*MOT_MOTthirteen + 26.25*MOT_MOTthree + 26.25*MOT_MOTtwelve + 26.25*MOT_MOTtwo + -15.0*VTO_VTOeight + -15.0*VTO_VTOeleven + -15.0*VTO_VTOfive + -15.0*VTO_VTOfour + -15.0*VTO_VTOfourteen + -15.0*VTO_VTOnine + -15.0*VTO_VTOseven + -15.0*VTO_VTOsix + -15.0*VTO_VTOten + -15.0*VTO_VTOthirteen + -15.0*VTO_VTOthree + -15.0*VTO_VTOtwelve + -15.0*VTO_VTOtwo + 1e-05*backlog_backlogWeek10 + 1e-05*backlog_backlogWeek11 + 1e-05*backlog_backlogWeek12 + 1e-05*backlog_backlogWeek13 + 1e-05*backlog_backlogWeek14 + 1e-05*backlog_backlogWeek2 + 1e-05*backlog_backlogWeek3 + 1e-05*backlog_backlogWeek4 + 1e-05*backlog_backlogWeek5 + 1e-05*backlog_backlogWeek6 + 1e-05*backlog_backlogWeek7 + 1e-05*backlog_backlogWeek8 + 1e-05*backlog_backlogWeek9 + 10000000

In [48]:
backlogDf

,week,backlogValue
0,0.0,0.0
0,1.0,0.0
0,2.0,0.0
0,3.0,0.0
0,4.0,0.0
0,5.0,0.0
0,6.0,0.0


In [30]:
totalCost

0

In [31]:
labor_cost(currentWeek)

201528.0

In [32]:
totalCost= totalCost+labor_cost(currentWeek)

In [43]:
print('total cost',totalCost+=labor_cost(currentWeek))

SyntaxError: invalid syntax (<ipython-input-43-a9f1ff65b924>, line 1)

In [37]:
A = 0
A += 1
A

1

In [40]:
totalCost+=labor_cost(currentWeek)

In [41]:
totalCost

604584.0